In [1]:
!pip install python-igraph leidenalg
#https://leidenalg.readthedocs.io/en/stable/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.7 MB/s eta 0:00:00


In [2]:
import igraph as ig
import pandas as pd

In [30]:
edges = pd.read_csv('cora.cites', sep='\t', header=None, names=['source', 'target'])
nodes = pd.read_csv('cora.content', sep='\t', header=None, names=['paper_id', 'word_attributes', 'label'])

In [37]:
all_node_ids = set(nodes['paper_id']).union(set(edges['source']), set(edges['target']))
node_ids = {node: idx for idx, node in enumerate(all_node_ids)}
inverse_mapping = {idx: node for node, idx in node_ids.items()}

In [38]:
g = ig.Graph(directed=False)
g.add_vertices(len(node_ids))
g.add_edges([(node_ids[src], node_ids[trg]) for src, trg in edges.values])

In [39]:
import leidenalg

partition = leidenalg.find_partition(g, leidenalg.ModularityVertexPartition)
communities = partition.membership

In [40]:
modularity_score = g.modularity(partition.membership)

print(f'Modularity Score: {modularity_score}')

Modularity Score: 0.8260720509956533


In [41]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

In [42]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(nodes['label'])

predicted_labels = communities